## Map Scraper
Use Selenium to scrape the bayern atlas map by iterating through every pixel on the map object, clicking there, ad collecting the HTML content from the popup.

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import MoveTargetOutOfBoundsException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.firefox import GeckoDriverManager

import time
import math
import numpy as np
import datetime


In [ ]:

STEP_SIZE = 1

filename = datetime.datetime.now().strftime("%Y%m%d%H%M%S")+"_Map_Scrape.txt"
profile= webdriver.FirefoxProfile()
profile.set_preference( "layout.css.devPixelsPerPx", "0.5" )
driver = webdriver.Firefox(
    firefox_profile=profile,
    executable_path=GeckoDriverManager().install()
    )
website_url = "https://geoportal.bayern.de/bauleitplanungsportal/karte.html"
driver.get(website_url)
time.sleep(20)
driver.switch_to.frame("map")
ol_viewport = driver.find_element(By.XPATH, '/html/body/div[2]/div[7]')
viewport_width = math.floor(ol_viewport.size['width'])
viewport_height =math.floor(ol_viewport.size['height'])
 
for x in np.arange(0, viewport_width, STEP_SIZE):
    for y in np.arange(0, viewport_height, STEP_SIZE):
        try:
            action = ActionChains(driver)
            action.move_to_element_with_offset(ol_viewport, x, y)
            action.click()
            action.perform()
            time.sleep(5)
            panel_body_html = driver.find_element(
                    By.CLASS_NAME, "panel-body"
                ).get_attribute("outerHTML")
            output_text = (
                "<coords value = "+
                str((x,y))+">\n"+
                panel_body_html+
                "\n<coords/>\n"
            )
        except MoveTargetOutOfBoundsException as e:
            output_text = (
            "<coords value = "+
            str((x,y))+">\n"+
            "Out of bounds"+
            "\n<coords/>\n"
            )
        except NoSuchElementException as e:
            output_text = (
            "<coords value = "+
            str((x,y))+">\n"+
            "No Element 'panel-body' found"+
            "\n<coords/>\n"
        )

        with open(filename, 'a', encoding="utf-8") as file:
            file.write(output_text +'\n')

driver.quit()